In [149]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

import datetime
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)

In [2]:
np.set_printoptions(suppress=True)

In [4]:
# from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

https://baseballsavant.mlb.com/leaderboard/statcast-park-factors?type=year&year=2022&batSide=&stat=index_wOBA&condition=All&rolling=no

In [5]:
#Selenium options
options = webdriver.ChromeOptions()

### Un-comment bellow if you want to run in headless mode ###
# options.add_argument("--headless")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

[WDM] - Downloading: 100%|█████████████████| 8.41M/8.41M [00:00<00:00, 43.4MB/s]


In [174]:
table_arr = []
for year in [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    url = f"https://baseballsavant.mlb.com/leaderboard/statcast-park-factors?type=year&year={year}&batSide=&stat=index_wOBA&condition=All&rolling=no"
    driver.get(url)

    headers = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'tr-component-row')))[0].text.split(' ')
    headers[4] = headers[4] + ' ' + headers[5]
    headers.remove('Factor')
    
    table = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tbody')))[0].text.split('\n')
    
#     driver.find_elements(By.TAG_NAME, 'tbody')[0].text.split('\n')

    
    for i in range(len(table)):
        table_split = table[i].split(' ')
        stats = [int(s.replace(',', '')) for s in table_split[-16:]]

        rank = int(table_split[0])
        team_info = team_info = [ii for ii in table_split if bool(re.match(r'[a-zA-Z]', ii))]

        if team_info[0] not in ['Red', 'White', 'Blue']:
            team = team_info[0]
            venue = ' '.join(team_info[1:])
        else:
            team = ' '.join(team_info[:2])
            venue = ' '.join(team_info[2:])

        team = team_info[0] + ' ' + team_info[1] if team_info[0] in ['Red', 'White', 'Blue'] else team_info[0]



        row = [rank] + [team] + [venue] + [year] + stats
        table_arr.append(row)

In [175]:
pd.DataFrame(table_arr, columns=headers)

,Rk.,Team,Venue,Year,Park Factor,wOBACon,xwOBACon,BACON,xBACON,HardHit,R,OBP,H,1B,2B,3B,HR,BB,SO,PA
0,1,Rockies,Coors Field,2018,118,121,107,118,106,105,139,114,119,109,143,184,127,97,94,6158
1,2,Rangers,Globe Life Park in Arlington,2018,113,111,104,108,102,104,128,110,112,106,114,163,127,106,89,6353
2,3,Nationals,Nationals Park,2018,108,108,105,107,104,104,117,105,107,102,123,72,120,102,96,6159
3,4,Yankees,Yankee Stadium,2018,106,105,102,105,103,102,112,106,108,114,92,83,112,106,96,6687
4,5,Red Sox,Fenway Park,2018,104,105,105,105,103,104,108,103,107,101,137,120,94,94,98,6774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,26,Guardians,Progressive Field,2022,96,95,98,96,98,99,92,97,94,98,90,55,91,100,104,6155
146,27,Athletics,Oakland Coliseum,2022,96,93,94,96,96,95,92,99,97,98,110,115,71,101,96,6075
147,28,Mets,Citi Field,2022,94,92,96,94,97,95,88,97,91,97,79,92,81,111,104,6174
148,29,Padres,Petco Park,2022,91,90,98,90,98,98,83,92,89,95,73,53,95,95,105,6756
